In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_kaution, get_stadtmitte, get_kueche, get_bad, get_mobliert, get_quadratmeter

In [15]:
file_path = '../../data/Wohnungen_2.csv'
data = pd.read_csv(file_path, sep=';')

bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'SingleHighIncome', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(
    lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

data = data[data['Bewohnerkategorie'] != 'Keine']

columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})

data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Entfernung zur Stadtmitte'] = data['Entfernung zur Stadtmitte'].apply(get_stadtmitte)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)
#Code Cleaning

C:\Users\cardu\AppData\Local\Temp\ipykernel_16868\4099255852.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1})


In [25]:
#Data Processing and Training
model = BayesianNetwork([
    ('Hausmeister', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie'),
    ('Garage', 'Bewohnerkategorie'),
    ('Alter', 'Bewohnerkategorie'),
    ('Aufzug', 'Bewohnerkategorie'),
    ('Entfernung zur Stadtmitte', 'Bewohnerkategorie'),
    ('Moebliert', 'Bewohnerkategorie'),
    ('Quadratmeter', 'Bewohnerkategorie')
])

# Training des Bayes-Netzes
model.fit(data, estimator=MaximumLikelihoodEstimator)
# Inferenz durchführen
inference = VariableElimination(model)


### Beispiele für die Inferenz
Wir vergleichen die Ergebnisse des Modells anhand der Beispiele "example_single_high_income", "example_single_high_income2", "example_single_high_income3". Hierbei wollen wir einmal ein Vergleich darstellen mit den Daten wo die Bewohnerkategorie "SingleHighIncome", Ja gesagt hat.

In [54]:
#Example Input with get methods from data_cleaning.text_to_numeric
example_single_high_income = {
    "Hausmeister": 0,  # Hausmeister vorhanden
    "Kindergarten": get_kindergarten("erreichbar"),  # Keine Relevanz für Single High Income
    "S-Bahn": get_bahn("nein"),  # Sehr nah an der S-Bahn
    "Garage": 1,  # Garage vorhanden
    "Alter": get_alter("ueber 100"),  # Neubau
    "Aufzug": 0,  # Aufzug vorhanden
    "Entfernung zur Stadtmitte": get_stadtmitte(">30km"),  # Zentrum aus den Daten
    "Moebliert": get_mobliert("nein"),  # Voll möbliert
    "Quadratmeter": get_stadtmitte("71-80")  # Beispiel aus Datei, Näherung
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_single_high_income)

# Wahrscheinlichkeiten in ein Dictionary umwandeln
result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

# Ausgabe schön formatiert untereinander
print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:.2%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 0.00%
Expatriate: 0.00%
Kleinfamilie: 0.00%
Kleinfamilie, DINK: 0.00%
SingleHighIncome: 100.00%
Studierende: 0.00%
Studierende, Rentnerpaar: 0.00%


In [52]:
example_single_high_income2 = {
    "Hausmeister": 0,  # Hausmeister vorhanden
    "Kindergarten": get_kindergarten("erreichbar"),  # Keine Relevanz für Single High Income
    "Aufzug": 0,  # Aufzug vorhanden
    "Entfernung zur Stadtmitte": get_stadtmitte(">30km"),  # Zentrum aus den Daten
    "Moebliert": get_mobliert("nein"),  # Voll möbliert
    "Quadratmeter": get_stadtmitte("71-80")  # Beispiel aus Datei, Näherung
}
# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_single_high_income2)

# Wahrscheinlichkeiten in ein Dictionary umwandeln
result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

# Ausgabe schön formatiert untereinander
print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:.2%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 14.20%
Expatriate: 14.20%
Kleinfamilie: 14.20%
Kleinfamilie, DINK: 14.20%
SingleHighIncome: 14.83%
Studierende: 14.20%
Studierende, Rentnerpaar: 14.20%


In [55]:
example_single_high_income3 = {
    "Hausmeister": 0,  # Hausmeister vorhanden
    "Kindergarten": get_kindergarten("erreichbar"),  # Keine Relevanz für Single High Income
    "S-Bahn": get_bahn("erreichbar"),  # Sehr nah an der S-Bahn
    "Garage": 0,  # Garage vorhanden
    "Alter": get_alter("51-75"),  # Neubau
    "Aufzug": 0,  # Aufzug vorhanden
    "Entfernung zur Stadtmitte": get_stadtmitte(">30km"),  # Zentrum aus den Daten
    "Moebliert": get_mobliert("nein"),  # Voll möbliert
    "Quadratmeter": get_stadtmitte("71-80")  # Beispiel aus Datei, Näherung
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_single_high_income3)

# Wahrscheinlichkeiten in ein Dictionary umwandeln
result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

# Ausgabe schön formatiert untereinander
print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:.2%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 14.29%
Expatriate: 14.29%
Kleinfamilie: 14.29%
Kleinfamilie, DINK: 14.29%
SingleHighIncome: 14.29%
Studierende: 14.29%
Studierende, Rentnerpaar: 14.29%


### Auswertung der Ergebnisse
Wie zu erwarten war, hat das erste Beispiel eine 100% Wahrscheinlichkeit für die Bewohnerkategorie "SingleHighIncome" ergeben. Das Zweite Beispiel hingegen hatte trotz drei gelöschter Wohnungskategorien, noch immer die Höchste Wahrscheinlichkeit für die Bewohnerkategorie "SingleHighIncome" hat. Mit der Änderung das "SingleHighIncome" nur noch eine Wahrscheinlichkeit von 14,83% hat. Hingegen bei dem dritten Beispiel, wo alle Merkmale wieder vorhanden sind, aber die drei Merkmale "S-Bahn", "Garage" und "Alter" geändert wurden, hat die Bewohnerkategorie "SingleHighIncome" nur noch eine Wahrscheinlichkeit von 14,29%.

### Potenzielle Erklärung
Die Wahrscheinlichkeiten der Bewohnerkategorien sind durch die fehlenden Merkmale im zweiten Beispiel gesunken. Aber mit den gleichen Merkmalen wie im ersten Beispiel, jedoch mit anderen Werten, hat die Bewohnerkategorie "SingleHighIncome" eine geringere Wahrscheinlichkeit. Das könnte daran liegen, dass die Merkmale "S-Bahn", "Garage" und "Alter" eine höhere Gewichtung haben als die anderen Merkmale für die Bewohnerkategorie "SingleHighIncome". Die Änderung dieser Merkmale hat die Wahrscheinlichkeiten der Bewohnerkategorien beeinflusst und zeigt, wie das Bayes-Netz die Beziehungen zwischen den Merkmalen und den Bewohnerkategorien modelliert. Den hierbei war es wichtiger das die Merkmale lieber nicht vorhanden sind, als das sie vorhanden sind, mit veränderten Werten.